In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.path import Path
from matplotlib.patches import PathPatch

from shapely.geometry import Polygon

import geopandas as gpd

import folium

from pyproj import Transformer

In [2]:


# WGS84 (lat/lon) <-> Web Mercator (meters)
to_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
to_4326 = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)

def project_to_3857(lon, lat):
    return to_3857.transform(lon, lat)

def project_to_4326(x, y):
    return to_4326.transform(x, y)

In [3]:
def bezier_curve(start, control1, control2, end, n=100):
    """Generate a cubic Bezier curve."""
    t = np.linspace(0, 1, n)[:, None]  # shape (n, 1)
    start, control1, control2, end = map(np.array, [start, control1, control2, end])
    curve = (
        (1 - t)**3 * start
        + 3 * (1 - t)**2 * t * control1
        + 3 * (1 - t) * t**2 * control2
        + t**3 * end
    )
    return curve


In [4]:
def perp(v):
    return np.array([-v[1], v[0]])  # rotate 90° left


def flow_patch_geo(
    start_ll, end_ll,
    start_width,
    end_width,
    color,
    bend=0.1,
    curvature=50,
    inherit_normal=None,
    start_offset_m=0,
    end_offset_m=0,
    offset_direction="left",  # "left", "right", or np.array([x,y])

):
    """
    Construct a flow patch polygon where width and offsets are expressed in meters.

    start_offset_m / end_offset_m:
        Lateral shift (in meters) applied to the start/end points.
        Positive values shift to the chosen offset_direction.
    """

    # ---- Project lat/lon to EPSG:3857 meters ----
    start = np.array(project_to_3857(*start_ll))
    end   = np.array(project_to_3857(*end_ll))

    # Direction vector in meters
    d = end - start
    d_norm = np.linalg.norm(d)
    if d_norm == 0:
        raise ValueError("Start and end coordinates are identical; cannot compute direction.")

    ud = d / d_norm  # unit direction

    # Determine base normal
    if inherit_normal is None:
        n = perp(ud)
        n = n / np.linalg.norm(n)
    else:
        n = inherit_normal / np.linalg.norm(inherit_normal)

    # ---- Determine offset direction ----
    if isinstance(offset_direction, str):
        if offset_direction == "left":
            offset_n = n
        elif offset_direction == "right":
            offset_n = -n
        else:
            raise ValueError("offset_direction must be 'left', 'right', or a 2D vector.")
    else:
        # custom vector
        offset_n = np.array(offset_direction)
        offset_n = offset_n / np.linalg.norm(offset_n)

    # ---- Apply lateral offsets in meters ----
    start = start + offset_n * start_offset_m
    end   = end   + offset_n * end_offset_m

    # ---- Bézier control points in meters ----
    #ctrl1 = start + d * bend + n * curvature
    #ctrl2 = end   - d * bend + n * curvature
    ctrl1 = start + 0.33 * d + n * (curvature * 0.05)   # mild, near start
    ctrl2 = start + 0.66 * d + n * (curvature * 1.00)   # strong, near midpoint

    # ---- Construct edges with width in meters ----
    sw = start_width / 2.0
    ew = end_width   / 2.0

    top = bezier_curve(
        start + n * sw,
        ctrl1 + n * sw,
        ctrl2 + n * ew,
        end   + n * ew
    )

    bottom = bezier_curve(
        end   - n * ew,
        ctrl2 - n * ew,
        ctrl1 - n * sw,
        start - n * sw
    )

    # Combine and close
    verts = np.vstack([top, bottom, top[0]])

    # Convert back to lat/lon
    verts_ll = [project_to_4326(x, y) for x, y in verts]

    width_and_color = [start_width, color]

    #return Polygon(verts_ll), n, width_and_color
    return Polygon(verts_ll), width_and_color

In [5]:
## list of lat longs 
'''
[(47.683729430644014, -122.34433918947462),
 (47.687259045590366, -122.34435249866571),
 (47.69108874059965, -122.34437859849116),
 (47.69480518468918, -122.34442577658399),
 (47.698157590581, -122.34452735842333),
 (47.70168220570689, -122.3445039810145),
 (47.70544935545624, -122.34455265309363),
 (47.71273742733618, -122.34473223501618),
 (47.7194691813143, -122.34484691221628),
 (47.723740055102255, -122.34489403611852)]

'''

'\n[(47.683729430644014, -122.34433918947462),\n (47.687259045590366, -122.34435249866571),\n (47.69108874059965, -122.34437859849116),\n (47.69480518468918, -122.34442577658399),\n (47.698157590581, -122.34452735842333),\n (47.70168220570689, -122.3445039810145),\n (47.70544935545624, -122.34455265309363),\n (47.71273742733618, -122.34473223501618),\n (47.7194691813143, -122.34484691221628),\n (47.723740055102255, -122.34489403611852)]\n\n'

In [6]:
## E line going from south to north
point_list = [
     (47.683729430644014, -122.34433918947462),
     (47.687259045590366, -122.34435249866571),
     (47.69108874059965, -122.34437859849116),
     (47.69480518468918, -122.34442577658399),
     (47.698157590581, -122.34452735842333)
]

rider_numbers = {
    0: [5, 0], #5
    1: [3, 2], #6
    2: [2, 4],  #4
    3: [2, 1]
}

In [7]:
#### make an ordered list that has the avg departing load at every stop

print (type(rider_numbers))
avg_departing_load = []

current_load = 0
for stop_num, rider_nums in rider_numbers.items():
    current_load = current_load + rider_nums[0]
    current_load = current_load - rider_nums[1]
    avg_departing_load.append(current_load)
    print (rider_nums)
    print (current_load)

print(avg_departing_load)

<class 'dict'>
[5, 0]
5
[3, 2]
6
[2, 4]
4
[2, 1]
5
[5, 6, 4, 5]


In [8]:
Starting_Y_Val = point_list[0][0]
Second_Y_Val = point_list[1][0]
Third_Y_Val = point_list[2][0]

Starting_X_Val = point_list[0][1]
Second_X_Val = point_list[1][1]
Third_X_Val = point_list[2][1]

In [9]:
WIDTH_MULTIPLIER = 50

polygon_list = []
width_and_color_list = []

In [10]:
point_list[0][0]

47.683729430644014

In [11]:
rider_numbers[1][0]

3

In [12]:
def make_trunk_polygon(segment_num, width):
    X_Val_1 = point_list[segment_num][1]
    Y_Val_1 = point_list[segment_num][0]
    X_Val_2 = point_list[segment_num + 1][1]
    Y_Val_2 = point_list[segment_num + 1][0]
    trunk, width_and_color = flow_patch_geo(
        start_ll = (X_Val_1, Y_Val_1),
        end_ll = (X_Val_2, Y_Val_2),
        start_width=width * WIDTH_MULTIPLIER,  # 40 meters
        end_width=width * WIDTH_MULTIPLIER,     # 40 meters
        start_offset_m = 0,
        end_offset_m = 0,
        bend = 0.1,
        curvature = 0,
        color = 'cornflowerblue'
    )     
    
    polygon_list.append(trunk)
    width_and_color_list.append(width_and_color)

In [13]:
def make_boarding_polygon(segment_num, width, start_offset, end_offset):
    X_Val_1 = point_list[segment_num][1]
    Y_Val_1 = point_list[segment_num][0]
    X_Val_2 = point_list[segment_num + 1][1]
    Y_Val_2 = point_list[segment_num + 1][0]
    trunk, width_and_color = flow_patch_geo(
        start_ll = (X_Val_1, Y_Val_1),
        end_ll = (X_Val_2, Y_Val_2),
        start_width=width * WIDTH_MULTIPLIER,  
        end_width=width * WIDTH_MULTIPLIER,   
        #start_offset_m = 150,    ## start offset should be large (2.5 * end offset?)
        #end_offset_m = 50,    ## end offset should be trunk_width * 0.5 + boarding_width * 0.5
        start_offset_m = start_offset,   
        end_offset_m = end_offset,
        bend = 0.1,
        curvature = 20,
        color = 'orange'
    )  

    polygon_list.append(trunk)
    width_and_color_list.append(width_and_color)

In [14]:
def make_alighting_polygon(segment_num, width, start_offset, end_offset):
    X_Val_1 = point_list[segment_num][1]
    Y_Val_1 = point_list[segment_num][0]
    X_Val_2 = point_list[segment_num + 1][1]
    Y_Val_2 = point_list[segment_num + 1][0]
    trunk, width_and_color = flow_patch_geo(
        start_ll = (X_Val_1, Y_Val_1),
        end_ll = (X_Val_2, Y_Val_2),
        start_width=width * WIDTH_MULTIPLIER,  
        end_width=width * WIDTH_MULTIPLIER,   
        #start_offset_m = -50,    ## start offset should be trunk_width * 0.5 + boarding_width * 0.5
        #end_offset_m = -150,    ## end offset should be large (2.5 * end offset?)
        start_offset_m = start_offset,    ## start offset should be trunk_width * 0.5 + boarding_width * 0.5
        end_offset_m = end_offset,    ## end offset should be large (2.5 * end offset?)
        bend = -0.1,
        curvature = -40,
        color = 'green'
    )  

    polygon_list.append(trunk)
    width_and_color_list.append(width_and_color)

In [15]:
for stop_num, rider_nums in rider_numbers.items():
    current_trunk_size = avg_departing_load[stop_num]
    current_boarders = rider_nums[0]
    current_alighters = rider_nums[1]
    
    alighting_offset = current_trunk_size * 0.5 * WIDTH_MULTIPLIER + current_alighters * 0.5 * WIDTH_MULTIPLIER
    #boarding_offset = current_trunk_size * WIDTH_MULTIPLIER + current_boarders * WIDTH_MULTIPLIER
    #alighting_offset = current_trunk_size * WIDTH_MULTIPLIER + current_alighters * WIDTH_MULTIPLIER
    if (stop_num == 0):
        make_trunk_polygon(stop_num, current_trunk_size)
    
    else:
        boarding_offset = avg_departing_load[stop_num-1] * 0.5 * WIDTH_MULTIPLIER + current_boarders * 0.5 * WIDTH_MULTIPLIER # declaring here so i can get previous stop num
        
        make_trunk_polygon(stop_num, current_trunk_size)
        make_boarding_polygon(stop_num - 1, current_boarders, boarding_offset * 2, boarding_offset)
        make_alighting_polygon(stop_num, current_alighters, -alighting_offset, -(alighting_offset * 2))
        
    
    print (rider_nums)

[5, 0]
[3, 2]
[2, 4]
[2, 1]


In [16]:
width_and_color_list

[[250, 'cornflowerblue'],
 [300, 'cornflowerblue'],
 [150, 'orange'],
 [100, 'green'],
 [200, 'cornflowerblue'],
 [100, 'orange'],
 [200, 'green'],
 [250, 'cornflowerblue'],
 [100, 'orange'],
 [50, 'green']]

In [17]:
#colors = ['cornflowerblue', 'orange', 'cornflowerblue', 'green']


m = folium.Map(location=[47.683729430644014, -122.34433918947462], zoom_start=14)



for poly, width_and_color in zip(polygon_list, width_and_color_list):
    color = width_and_color[1]
    
    if(color == 'cornflowerblue'):
        tt = (f"Departing Load = {width_and_color[0] / WIDTH_MULTIPLIER}") ## figure out how to get values from the method calls
    elif(color == 'orange'):
        tt = (f"Riders boarding = {width_and_color[0] / WIDTH_MULTIPLIER}")
    elif(color == 'green'):
        tt = (f"Riders alighting = {width_and_color[0] / WIDTH_MULTIPLIER}")
    folium.GeoJson(
        poly,
        style_function=lambda _, col = color: {
        #style_function=lambda _, col = : {
            "fillColor": col,
            "color": col,
            "weight": 1,
            "fillOpacity": 0.7
        }, 
        tooltip = tt
    ).add_to(m)

m